#  **This is a project for week 3**

## installing and importing necessary libraries

In [0]:
#!pip install lxml
#!pip install html5lib
#!pip install BeautifulSoup4
import pandas as pd
import html5lib
from geopy.geocoders import Nominatim
import requests
import numpy as np
from sklearn.cluster import KMeans

## importing and manipulating dataset. Task 1

In [0]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [0]:
df = df.drop(df[df['Borough'] == 'Not assigned'].index)

In [0]:
df = df.rename(columns={'Postcode': 'PostalCode'})

In [0]:
df.reset_index(drop=True, inplace=True);

In [19]:
df.shape

(210, 3)

In [20]:
df.head(2)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village


## inserting log and lan; Task 2.

In [0]:
df1=pd.read_csv('https://cocl.us/Geospatial_data')

In [0]:
lat=[]
for i in range(210):
  a=df.loc[i,['PostalCode']]
  b=df1[df1['Postal Code']==a[0]]['Latitude']
  lat.append(b.values[0])

In [0]:
lon=[]
for i in range(210):
  a=df.loc[i,['PostalCode']]
  b=df1[df1['Postal Code']==a[0]]['Longitude']
  lon.append(b.values[0])

In [0]:
df['Latitude']=lat
df['Longitude']=lon

In [25]:
df[df['PostalCode']=='M4B'] # same results as in example

,PostalCode,Borough,Neighborhood,Latitude,Longitude
10,M4B,East York,Woodbine Gardens,43.706397,-79.309937
11,M4B,East York,Parkview Hill,43.706397,-79.309937


## Task 3; clustering

In [0]:
CLIENT_ID = 'YHLSQYTPNMPEPCCPAMR2NBXPTD1LVTXVDX0ZDSJCAUAZTPKY'
CLIENT_SECRET = '1MFFUIP4C4XVZC4HX2GJ2HHUJ0NJIVM5SGQUK3LOH4CEI2P5'
VERSION = '20191231'

In [0]:
#create a dataframe with only the boroughs that have the word toronto
dff=df[df['Borough']=="Downtown Toronto" ]

In [46]:
#find the adrees of Downtown Toronto

address='Downtown Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [0]:
# using the function in the excericise
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
toronto_venues = getNearbyVenues(dff['Neighborhood'],dff['Latitude'],dff['Longitude'])

In [0]:
#creating the final df
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert(0,'Neighborhoods',toronto_venues['Neighborhood'])

In [0]:
#now we want the most common
 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()


In [93]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Restaurant,Burger Joint,Gym
1,Bathurst Quay,Airport Terminal,Airport Lounge,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Service
2,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Café,Farmers Market,Seafood Restaurant,Beer Bar,Bakery,Cheese Shop,Comfort Food Restaurant
3,CN Tower,Airport Terminal,Airport Lounge,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Service
4,Cabbagetown,Restaurant,Coffee Shop,Café,Italian Restaurant,Pizza Place,Pub,Bakery,Park,Playground,Beer Store


In [0]:
# and now clustering
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [0]:
#visualazation; importing libraries
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [0]:
#add lat and lan on the above df
# add clustering labels

toronto_merged = dff

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

In [115]:
# finish it off

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters